In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import datetime
import geocoder
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',50)
pd.set_option('max_colwidth',200)
%matplotlib inline
import os
from itertools import chain 
from pandas.core.frame import DataFrame
import seaborn as sns
def s2tw(ori_str):
    ori_str_lv = ori_str
    conv_h = opencc.OpenCC('s2tw')  # convert from Simplified Chinese to Traditional Chinese + Taiwan phrase
    conv_str_lv = conv_h.convert(ori_str_lv)
    return conv_str_lv
def s2t(ori_str):
    ori_str_lv = ori_str
    conv_h = opencc.OpenCC('s2t')  # convert from Simplified Chinese to Traditional Chinese
    conv_str_lv = conv_h.convert(ori_str_lv)
    return conv_str_lv
def t2s(ori_str):
    ori_str_lv = ori_str
    conv_h = opencc.OpenCC('t2s')  # convert from Traditional Chinese to Simplified Chinese
    conv_str_lv = conv_h.convert(ori_str_lv)
    return conv_str_lv

# 專利資料(授權)

In [2]:
os.chdir(r'E:\paper\IncoPat\發明授權')
os.getcwd()

'E:\\paper\\IncoPat\\發明授權'

In [3]:
files = os.listdir(os.getcwd())
files.sort() 
if '.DS_Store' in files:
    files = files[1:]
    
cdataI = DataFrame()
n = 1
for i in files:
    data = pd.read_excel(i)
    data.drop({'序号'},axis=1,inplace=True) 
    cdataI = cdataI.append(data)
    print(' \'%s\' Attach (%s/%s)' % (i,n,len(files)))
    n = n+1

 '2020-03-14-1.xls' Attach (1/61)
 '2020-03-14-2.xls' Attach (2/61)
 '2020-03-14-3.xls' Attach (3/61)
 '2020-03-14-4.xls' Attach (4/61)
 '2020-03-14-5.xls' Attach (5/61)
 '2020-03-16-1.xls' Attach (6/61)
 '2020-03-16-2.xls' Attach (7/61)
 '2020-03-16-3.xls' Attach (8/61)
 '2020-03-16-4.xls' Attach (9/61)
 '2020-03-16-5.xls' Attach (10/61)
 '2020-03-17-1.xls' Attach (11/61)
 '2020-03-17-2.xls' Attach (12/61)
 '2020-03-17-3.xls' Attach (13/61)
 '2020-03-17-4.xls' Attach (14/61)
 '2020-03-17-5.xls' Attach (15/61)
 '2020-03-18-1.xls' Attach (16/61)
 '2020-03-18-2.xls' Attach (17/61)
 '2020-03-18-3.xls' Attach (18/61)
 '2020-03-18-4.xls' Attach (19/61)
 '2020-03-18-5.xls' Attach (20/61)
 '2020-03-19-1.xls' Attach (21/61)
 '2020-03-19-2.xls' Attach (22/61)
 '2020-03-19-3.xls' Attach (23/61)
 '2020-03-19-4.xls' Attach (24/61)
 '2020-03-19-5.xls' Attach (25/61)
 '2020-03-20-1.xls' Attach (26/61)
 '2020-03-20-2.xls' Attach (27/61)
 '2020-03-20-3.xls' Attach (28/61)
 '2020-03-20-4.xls' Attach (2

In [4]:
Ilen = len(cdataI)
cdataI_dup = cdataI.drop_duplicates(subset=None, keep='first', inplace= False)
Ilen_dup = len(cdataI_dup)
print('重複筆數:',Ilen - Ilen_dup)
print('發明授權專利 總筆數(已刪重複):',Ilen_dup)

重複筆數: 42620
發明授權專利 總筆數(已刪重複): 3596105


In [5]:
cdataI_dup.index = np.arange(Ilen_dup)  ### 修改index

In [6]:
dupnumber = cdataI_dup[cdataI_dup.duplicated('公开（公告）号',keep='first')]['公开（公告）号']
lendup = len(dupnumber)
print('公开（公告）号 重複筆數:',lendup)

公开（公告）号 重複筆數: 71


In [9]:
n=0
### 檢查哪個欄位不同 第n筆重複號
dupindex = cdataI_dup[cdataI_dup['公开（公告）号']==dupnumber.iloc[n]].index
for i in cdataI_dup.columns:
    if (cdataI_dup.loc[dupindex[0],'%s'%i]==cdataI_dup.loc[dupindex[1],'%s'%i]) == False:
        print('%s:'%i)
        print('     first:',cdataI_dup.loc[dupindex[0],'%s'%i])
        print('     second:',cdataI_dup.loc[dupindex[1],'%s'%i])

标准化申请人:
     first: nan
     second: nan
中国申请人地市:
     first: nan
     second: nan
中国申请人区县:
     first: nan
     second: nan
扩展同族:
     first: AU2013279604A1; JP2015529633A; US20150132764A1; IN10561DELN2014A; AU2013279601A1; BRPI1431958A2; CA2876872A1; JP6654432B2; US10359433B2; CN104507965B; BRPI1431957A2; CN104507965A; WO2013189775A1; KR1020150036102A; WO2013189778A1; IN10562DELN2014A; EP2861754A1; IN10562DELNP2014A; CN104619861A; IN10561DELNP2014A; AU2013279604B2; EP2676966A1; JP2015521475A; EP2861618A1; US20150147767A1; CA2876922A1; KR1020150036111A
     second: EP2861618B1; CN104619861A; EP2861618A1; US20150147767A1; CA2876922A1; AU2013279601A1; IN10561DELN2014A; BRPI1431957A2; WO2013189775A1; EP2861754A1; KR1020150036102A; AU2013279604B2; JP2015521475A; US20150132764A1; EP2676966A1; IN10561DELNP2014A; WO2013189778A1; IN10562DELN2014A; JP2015529633A; AU2013279604A1; CN104507965A; US10359433B2; CN104507965B; KR1020150036111A; BRPI1431958A2; JP6654432B2; CA2876872A1; IN10562DELNP201

In [7]:
### 只留最新一筆重複公開號
cdataI_dup = cdataI_dup.drop_duplicates('公开（公告）号',keep='last')
print('發明授權專利 總筆數(已刪重複公吿號):',len(cdataI_dup))

發明授權專利 總筆數(已刪重複公吿號): 3596034


In [8]:
print(cdataI_dup.groupby('公开国别').size())

公开国别
CN    3596034
dtype: int64


In [9]:
cdataI_dup.columns

Index(['标题', '文献页数', '合享价值度', '链接到incoPat', '公开（公告）号', '公开（公告）日', '申请号', '申请日',
       '申请人', '标准化申请人', '申请人国别代码', '申请人省市代码', '中国申请人地市', '中国申请人区县', '申请人地址',
       '申请人类型', '优先权信息', '权利要求数量', '首次公开日', '授权公告日', '专利类型', '发明人', 'IPC',
       'CPC', '简单同族', '扩展同族', 'inpadoc同族', '同族国家', '简单同族个数', '扩展同族个数',
       'inpadoc同族个数', '引证专利', '被引证专利', '被引证次数', '引证科技文献', '专利寿命（月）',
       '被引证国别(forward)', '失效日', '公开国别'],
      dtype='object')

In [8]:
cdataI_cn = cdataI_dup[cdataI_dup['申请人国别代码'] == '中国']
print(cdataI_cn.shape)

(2474719, 39)


In [9]:
cdataI_cn = cdataI_cn[~cdataI_cn['申请人省市代码'].isna()]
print(cdataI_cn.shape)

(2474718, 39)


# 區域代碼

In [13]:
city_code = pd.read_excel('C:/Users/user/Dropbox/paper/科技園區/China/省地縣碼對照_new.xls')
city_code.fillna('',inplace=True)
city_code['代碼'] = city_code['代碼'].map(lambda x:str(int(x)))
city_code.fillna('',inplace=True)
city_code['行政區'] = city_code.apply(lambda x:x['省份']+x['市級']+x['名稱'],axis=1)

In [152]:
city_code

,代碼,省份,市級,名稱,行政區
0,110101,北京市,,东城区,北京市东城区
1,110102,北京市,,西城区,北京市西城区
2,110103,北京市,,崇文区,北京市崇文区
3,110104,北京市,,宣武区,北京市宣武区
4,110105,北京市,,朝阳区,北京市朝阳区
5,110106,北京市,,丰台区,北京市丰台区
6,110107,北京市,,石景山区,北京市石景山区
7,110108,北京市,,海淀区,北京市海淀区
8,110109,北京市,,门头沟区,北京市门头沟区
9,110111,北京市,,房山区,北京市房山区


In [14]:
city_list = []
for i in list(city_code['省份'].unique()):
    city_list.append(list(city_code[city_code['省份']=='{}'.format(i)]['行政區'].unique()))

In [15]:
##'台湾','香港','澳门'不包含
def similaraity(city_list,prov,place):
    if prov=='北京':
        match = city_list[0]
    elif prov=='天津':
        match = city_list[1]
    elif prov=='河北':
        match = city_list[2]
    elif prov=='山西':
        match = city_list[3]
    elif prov=='内蒙古':
        match = city_list[4]
    elif prov=='辽宁':
        match = city_list[5]
    elif prov=='吉林':
        match = city_list[6]
    elif prov=='黑龙江':
        match = city_list[7]
    elif prov=='上海':
        match = city_list[8]
    elif prov=='江苏':
        match = city_list[9]
    elif prov=='浙江':
        match = city_list[10]
    elif prov=='安徽':
        match = city_list[11]
    elif prov=='福建':
        match = city_list[12]
    elif prov=='江西':
        match = city_list[13]
    elif prov=='山东':
        match = city_list[14]
    elif prov=='河南':
        match = city_list[15]
    elif prov=='湖北':
        match = city_list[16]
    elif prov=='湖南':
        match = city_list[17]
    elif prov=='广东':
        match = city_list[18]
    elif prov=='广西':
        match = city_list[19]
    elif prov=='海南':
        match = city_list[20]
    elif prov=='重庆':
        match = city_list[21]
    elif prov=='四川':
        match = city_list[22]
    elif prov=='贵州':
        match = city_list[23]
    elif prov=='云南':
        match = city_list[24]
    elif prov=='西藏':
        match = city_list[25]
    elif prov=='陕西':
        match = city_list[26]
    elif prov=='甘肃':
        match = city_list[27]
    elif prov=='青海':
        match = city_list[28]
    elif prov=='宁夏':
        match = city_list[29]
    elif prov=='新疆':
        match = city_list[30]
    else:
        return ''
    num = []
    for k in match:
        num.append(string_similar(place,k))
    return match[num.index(max(num))]

# 篩有申請專利的廠商名單

In [11]:
cdataI_cn = cdataI_cn[~cdataI_cn['中国申请人区县'].isna()]
print(cdataI_cn.shape)
cdataI_cn = cdataI_cn[cdataI_cn['申请人类型'].str.contains('企业')]
print(cdataI_cn.shape)
cdataI_cn['申请年份'] = cdataI_cn['申请日'].map(lambda x:int(str(x)[0:4]))

(2327146, 39)
(1364867, 39)


In [14]:
print(cdataI_cn[cdataI_cn['申请人类型']=='企业'].shape)
print(cdataI_cn[cdataI_cn['申请人类型']!='企业'].shape)

(1293996, 40)
(70871, 40)


In [15]:
applicant = cdataI_cn['申请人'].str.split(';',expand=True).stack().reset_index(level=1,drop=True).rename('申请人2')
cdataI_cn = cdataI_cn.join(applicant)
cdataI_cn['類別'] = cdataI_cn['申请人类型'].map(lambda x:x.strip().split('  '))
cdataI_cn = cdataI_cn[~cdataI_cn['申请号'].duplicated()]
cdataI_cn['申请人类型'] = cdataI_cn.apply(lambda x:x['類別'][0],axis=1)
cdataI_cn = cdataI_cn[cdataI_cn['申请人类型']=='企业']

In [16]:
print(cdataI_cn[cdataI_cn['申请人类型']=='企业'].shape)
print(cdataI_cn[cdataI_cn['申请人类型']!='企业'].shape)

(1327263, 42)
(0, 42)


In [18]:
app_list = list(cdataI_cn['申请人2'].unique())
app_df = pd.DataFrame({'申請人':app_list})

In [20]:
app_df.to_excel(r'E:\paper\專利授權申請人名單.xlsx')

# 地區編碼

In [17]:
cdataI_cn['申请人省市代码'].fillna('',inplace=True)
cdataI_cn['申请人省市代码'] = cdataI_cn['申请人省市代码'].map(lambda x:x.split(';')[0])
cdataI_cn['中国申请人地市'].fillna('',inplace=True)
cdataI_cn['中国申请人地市'] = cdataI_cn['中国申请人地市'].map(lambda x:re.sub(r"(\[)|(\])","",x))
cdataI_cn['中国申请人区县'].fillna('',inplace=True)
cdataI_cn['中国申请人区县'] = cdataI_cn['中国申请人区县'].map(lambda x:re.sub(r"(\[)|(\])","",x))
cdataI_cn['行政區'] = cdataI_cn.apply(lambda x:x['申请人省市代码']+x['中国申请人地市']+x['中国申请人区县'],axis=1)

In [18]:
#模糊比對行政區位置
cdataI_cn['校正'] = cdataI_cn.apply(lambda x:similaraity(city_list,x['申请人省市代码'],x['行政區']),axis=1)

In [19]:
cdataI_cn = pd.merge(cdataI_cn,city_code[['代碼','行政區']],left_on='校正',right_on='行政區',how='left')

In [28]:
cdataI_cn
##.to_csv(r'E:\paper\專利授權.csv')

,标题,文献页数,合享价值度,链接到incoPat,公开（公告）号,公开（公告）日,申请号,申请日,申请人,标准化申请人,申请人国别代码,申请人省市代码,中国申请人地市,中国申请人区县,申请人地址,申请人类型,优先权信息,权利要求数量,首次公开日,授权公告日,专利类型,发明人,IPC,CPC,简单同族,扩展同族,inpadoc同族,同族国家,简单同族个数,扩展同族个数,inpadoc同族个数,引证专利,被引证专利,被引证次数,引证科技文献,专利寿命（月）,被引证国别(forward),失效日,公开国别,申请年份,行政區_x,校正,代碼,行政區_y
0,扭矩传动组合装置及联轴器,14.0,9,到incoPat中查看\r\nCN103307125B,CN103307125B,2019-12-31,CN201210061572.1,2012-03-09,德昌电机(深圳)有限公司,[德昌电机-C],中国,广东,深圳,宝安,518125 广东省深圳市宝安区沙井镇新二工业村,企业,NaN,11,2013-09-18,2019-12-31,发明授权,赵凯; 聂金安; 周海强,F16D1/112; F16D3/68,F16D1/10; F16D3/68; F16D2001/102; F16D11/00,DE102013102387A1; US20130233669A1; CN103307125A; CN103307125B; US8814710B2,CN103307125B; US8814710B2; US20130233669A1; CN103307125A; DE102013102387A1,CN103307125A; US8814710B2; DE102013102387A1; US20130233669A1; CN103307125B,"DE, US, CN",5,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CN,2012,广东深圳宝安,广东省深圳市宝安区,440306,广东省深圳市宝安区
1,一种多模终端信息处理方法及多模终端,25.0,8,到incoPat中查看\r\nCN103369615B,CN103369615B,2019-12-31,CN201310013755.0,2013-01-15,中兴通讯股份有限公司,[中兴公司-C],中国,广东,深圳,南山,518057 广东省深圳市南山区高新技术产业园科技南路中兴通讯大厦法务部,企业,CN2012100987777 20120406; CN2012101887102 20120608,8,2013-10-23,2019-12-31,发明授权,余媛芳; 陆婷,H04W36/14; H04W48/18; H04W76/10; H04W88/06,H04W48/18; H04W76/25; H04W88/06; H04W36/14,CN103369615A; CN103369615B,WO2013149568A1; IN7302CHENP2014A; CN103369615B; US9769722B2; IN320641A1; CN103369615A; US20150063297A1; JP2015515815A,CN103369615B; US9769722B2; WO2013149568A1; CN103369615A; US20150063297A1; JP2015515815A,CN,2,8,6,CN102143560A; US20100128608A1; CN102197684A,NaN,NaN,NaN,NaN,NaN,NaN,CN,2013,广东深圳南山,广东省深圳市南山区,440305,广东省深圳市南山区
2,一种去除光阻蚀刻残留物的清洗液,7.0,9,到incoPat中查看\r\nCN103809394B,CN103809394B,2019-12-31,CN201210451687.1,2012-11-12,安集微电子科技(上海)股份有限公司,NaN,中国,上海,上海,浦东新,201201 上海市浦东新区华东路5001号金桥出口加工区(南区)T6-9幢底层,企业,NaN,8,2014-05-21,2019-12-31,发明授权,刘兵; 彭洪修; 孙广胜; 颜金荔,G03F7/42,G03F7/423; G03F7/425; G03F7/426,WO2014071688A1; CN103809394A; CN103809394B; TW201418452A,WO2014071688A1; TW201418452A; CN103809394B; CN103809394A,CN103809394B; CN103809394A; WO2014071688A1; TW201418452A,"WO, CN, TW",4,4,4,CN1875325A; US5968848A; CN101597548A; CN1659480A; CN1954267A,NaN,NaN,NaN,NaN,NaN,NaN,CN,2012,上海上海浦东新,上海市浦东新区,310115,上海市浦东新区
3,一种A柱盖板、A柱上饰板、仪表台的组合结构,7.0,7,到incoPat中查看\r\nCN103935307B,CN103935307B,2019-12-31,CN201410179205.0,2014-04-29,安徽江淮汽车集团股份有限公司,NaN,中国,安徽,合肥,蜀山,230601 安徽省合肥市桃花工业园始信路669号,企业,NaN,4,2014-07-23,2019-12-31,发明授权,任婷婷,B60R13/02,NaN,CN103935307A; CN103935307B,CN103935307B; CN103935307A,CN103935307A; CN103935307B,CN,2,2,2,CN203793248U; CN201881914U; CN1966314A; GB2355438A; CN201784695U; US20020024206A1,NaN,NaN,NaN,NaN,NaN,NaN,CN,2014,安徽合肥蜀山,安徽省合肥市蜀山区,340104,安徽省合肥市蜀山区
4,信道测量导频的指示信令的确定、CSI反馈方法及装置,21.0,10,到incoPat中查看\r\nCN104038312B,CN104038312B,2019-12-31,CN201310074674.1,2013-03-08,中兴通讯股份有限公司,[中兴公司-C],中国,广东,深圳,南山,518057 广东省深圳市南山区科技南路55号,企业,NaN,12,2014-09-10,2019-12-31,发明授权,陈艺戬; 戴博; 李儒岳; 孙云锋; 郭森宝,H04L1/00; H04L1/16,H04W24/10; H04W76/27; H04L5/0048; H04W72/042; H04W72/14; H04W88/02; H04W88/08,WO2014135100A1; CN104038312A; EP2966931A1; EP2966931A4; US10425850B2; US20160029238A1; CN104038312B; US20200008090A1,EP2966931A4; WO2014135100A1; CN104038312B; EP2966931A1; CN104038312A; US20200008090A1; US10425850B2; US20160029238A1,US20160029238A1; US10425850B2; CN104038312A; WO2014135100A1; EP2966931A4; US20200008090A1; EP2966931A1; CN104038312B,"WO, CN, EP, US",8,8,8,US20120208547A1; CN102696183A; WO2013021531A1,NaN,0.0,"<text>Signaling for PDSCH RE mapping and quasi co-location in DCI format 2D; Huawei, HiSilicon; 《3GPP TSG RAN WG1 Meeting #71 R1-124692》; 20121116; 第1-2页</text>",NaN,NaN,NaN,CN,2013,广东深圳南山,广东省深圳市南山区,440305,广东省深圳市南山区
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364862,磷矿化学分离法,6.0,8,到incoPat中查看\r\nCN85100187B,CN85100187B,1985-09-10,CN85100187.4,1985-04-01,陕西省化学肥料工业公司,NaN,中国,陕西,西安,雁塔,陕西省西安市雁塔路22号,企业,CN85100187 19850401,14,1985-09-10,1986-01-10,发明授权,路文兴,C01B25/01,C01B25/01; C01F5/24; C0

In [30]:
#申請數
counts = cdataI_cn.groupby(['代碼','申请年份'])['申请号'].count()

In [31]:
#引用數
cdataI_cn['被引证次数'].fillna(0,inplace=True)
cdataI_cn['权利要求数量'].fillna(0,inplace=True)
cite = cdataI_cn.groupby(['代碼','申请年份'])[['被引证次数','权利要求数量']].sum()

In [34]:
patent_data = pd.merge(cite,counts,left_index=True,right_index=True,how='outer')

In [35]:
patent_data.to_excel(r'E:\paper\專利授權_統計數量.xlsx')

# 專利資料(實用新型) P1~P3
數量過多分3次做

In [2]:
# 有專利授權廠商才保留
app_df = pd.read_excel(r'E:\paper\專利授權申請人名單.xlsx',index_col=0)
app_list = list(app_df['申請人'])

In [3]:
app_list

['德昌电机(深圳)有限公司',
 '中兴通讯股份有限公司',
 '安集微电子科技(上海)股份有限公司',
 '安徽江淮汽车集团股份有限公司',
 '江苏奥赛康药业有限公司',
 '中芯国际集成电路制造(上海)有限公司',
 '网神信息技术(北京)股份有限公司',
 '北京京东尚科信息技术有限公司',
 '上海博达数据通信有限公司',
 '京东方科技集团股份有限公司',
 '深圳汇生通科技股份有限公司',
 '杭州阿尔法红外检测技术有限公司',
 '海尔优家智能科技(北京)有限公司',
 '鲁南制药集团股份有限公司',
 '深圳桑菲消费通信有限公司',
 '未名兴旺系统作物设计前沿实验室(北京)有限公司',
 '南京朗洁环保科技有限公司',
 '深圳安诺盛科技有限公司',
 '合肥鑫晟光电科技有限公司',
 '上海有德化工科技有限公司',
 '合肥京东方光电科技有限公司',
 '中国建材国际工程集团有限公司',
 '百度在线网络技术(北京)有限公司',
 '东莞东阳光科研发有限公司',
 '富顶精密组件(深圳)有限公司',
 '广东小天才科技有限公司',
 '深圳市钱海网络技术有限公司',
 '上海翔琼生物技术有限公司',
 '北京三星通信技术研究有限公司',
 '中国航空工业集团公司沈阳发动机设计研究所',
 '福建天晴数码有限公司',
 '宁波舜宇光电信息有限公司',
 '杭州朗和科技有限公司',
 '青岛海尔空调器有限总公司',
 '湖北鼎龙化学股份有限公司',
 '北京盛诺基医药科技股份有限公司',
 '武汉华星光电技术有限公司',
 '腾讯科技(深圳)有限公司',
 '亳州易泽信息科技有限公司',
 '昆山德拉特兰传动科技有限公司',
 '湖南千盟物联信息技术有限公司',
 '成都秦川物联网科技股份有限公司',
 '无锡顶点医疗器械有限公司',
 '湖北中南勘察基础工程有限公司',
 '青岛海尔智能家电科技有限公司',
 '苏州鼎德电环保科技有限公司',
 '深圳华大智造科技有限公司',
 '北京奇虎科技有限公司',
 '四川新明塑业有限公司',
 '四川云盾光电科技有限公司',
 '华远医药研究院有限公司',
 '青岛海尔智能技术研发有限公司',
 '珠海格力电器股份有限公司',
 '佛山市顺德区美的洗涤电器制造有限公司',
 '深圳市圣必智科技开发有限

In [ ]:
city_code = pd.read_excel('C:/Users/user/Dropbox/paper/科技園區/China/省地縣碼對照_new.xls')
city_code.fillna('',inplace=True)
city_code['代碼'] = city_code['代碼'].map(lambda x:str(int(x)))
city_code.fillna('',inplace=True)
city_code['行政區'] = city_code.apply(lambda x:x['省份']+x['市級']+x['名稱'],axis=1)
city_list = []
for i in list(city_code['省份'].unique()):
    city_list.append(list(city_code[city_code['省份']=='{}'.format(i)]['行政區'].unique()))

In [ ]:
##'台湾','香港','澳门'不包含
def similaraity(city_list,prov,place):
    if prov=='北京':
        match = city_list[0]
    elif prov=='天津':
        match = city_list[1]
    elif prov=='河北':
        match = city_list[2]
    elif prov=='山西':
        match = city_list[3]
    elif prov=='内蒙古':
        match = city_list[4]
    elif prov=='辽宁':
        match = city_list[5]
    elif prov=='吉林':
        match = city_list[6]
    elif prov=='黑龙江':
        match = city_list[7]
    elif prov=='上海':
        match = city_list[8]
    elif prov=='江苏':
        match = city_list[9]
    elif prov=='浙江':
        match = city_list[10]
    elif prov=='安徽':
        match = city_list[11]
    elif prov=='福建':
        match = city_list[12]
    elif prov=='江西':
        match = city_list[13]
    elif prov=='山东':
        match = city_list[14]
    elif prov=='河南':
        match = city_list[15]
    elif prov=='湖北':
        match = city_list[16]
    elif prov=='湖南':
        match = city_list[17]
    elif prov=='广东':
        match = city_list[18]
    elif prov=='广西':
        match = city_list[19]
    elif prov=='海南':
        match = city_list[20]
    elif prov=='重庆':
        match = city_list[21]
    elif prov=='四川':
        match = city_list[22]
    elif prov=='贵州':
        match = city_list[23]
    elif prov=='云南':
        match = city_list[24]
    elif prov=='西藏':
        match = city_list[25]
    elif prov=='陕西':
        match = city_list[26]
    elif prov=='甘肃':
        match = city_list[27]
    elif prov=='青海':
        match = city_list[28]
    elif prov=='宁夏':
        match = city_list[29]
    elif prov=='新疆':
        match = city_list[30]
    else:
        return ''
    num = []
    for k in match:
        num.append(string_similar(place,k))
    return match[num.index(max(num))]

In [4]:
os.chdir(r'E:\paper\IncoPat\實用新型')
os.getcwd()

'E:\\paper\\IncoPat\\實用新型'

In [5]:
files = os.listdir(os.getcwd())
files.sort() 
if '.DS_Store' in files:
    files = files[1:]
k = files
files = []
for i in k:
    if '._' not in i:
        files.append(i)
files1 = files[0:60]
files2 = files[60:120]
files3 = files[120:170]
#P1
cdataU = DataFrame()
n = 1
for i in files1:
    data = pd.read_excel(i)
    data.drop({'序号'},axis=1,inplace=True) 
    cdataU = cdataU.append(data)
    print(' \'%s\' Attach (%s/%s)' % (i,n,len(files1)))
    n = n+1

 '2020-04-02-1.xls' Attach (1/60)
 '2020-04-02-2.xls' Attach (2/60)
 '2020-04-02-3.xls' Attach (3/60)
 '2020-04-02-4.xls' Attach (4/60)
 '2020-04-02-5.xls' Attach (5/60)
 '2020-04-03-1.xls' Attach (6/60)
 '2020-04-03-2.xls' Attach (7/60)
 '2020-04-03-3.xls' Attach (8/60)
 '2020-04-03-4.xls' Attach (9/60)
 '2020-04-03-5.xls' Attach (10/60)
 '2020-04-04-1.xls' Attach (11/60)
 '2020-04-04-2.xls' Attach (12/60)
 '2020-04-04-3.xls' Attach (13/60)
 '2020-04-04-4.xls' Attach (14/60)
 '2020-04-04-5.xls' Attach (15/60)
 '2020-04-05-1.xls' Attach (16/60)
 '2020-04-05-2.xls' Attach (17/60)
 '2020-04-05-3.xls' Attach (18/60)
 '2020-04-05-4.xls' Attach (19/60)
 '2020-04-05-5.xls' Attach (20/60)
 '2020-04-06-1.xls' Attach (21/60)
 '2020-04-06-2.xls' Attach (22/60)
 '2020-04-06-3.xls' Attach (23/60)
 '2020-04-06-4.xls' Attach (24/60)
 '2020-04-06-5.xls' Attach (25/60)
 '2020-04-07-1.xls' Attach (26/60)
 '2020-04-07-2.xls' Attach (27/60)
 '2020-04-07-3.xls' Attach (28/60)
 '2020-04-07-4.xls' Attach (2

In [13]:
#P2
cdataU = DataFrame()
n = 1
for i in files2:
    data = pd.read_excel(i)
    data.drop({'序号'},axis=1,inplace=True) 
    cdataU = cdataU.append(data)
    print(' \'%s\' Attach (%s/%s)' % (i,n,len(files2)))
    n = n+1

 '2020-04-14-1.xls' Attach (1/60)
 '2020-04-14-2.xls' Attach (2/60)
 '2020-04-14-3.xls' Attach (3/60)
 '2020-04-14-4.xls' Attach (4/60)
 '2020-04-14-5.xls' Attach (5/60)
 '2020-04-15-1.xls' Attach (6/60)
 '2020-04-15-2.xls' Attach (7/60)
 '2020-04-15-3.xls' Attach (8/60)
 '2020-04-15-4.xls' Attach (9/60)
 '2020-04-15-5.xls' Attach (10/60)
 '2020-04-16-1.xls' Attach (11/60)
 '2020-04-16-2.xls' Attach (12/60)
 '2020-04-16-3.xls' Attach (13/60)
 '2020-04-16-4.xls' Attach (14/60)
 '2020-04-16-5.xls' Attach (15/60)
 '2020-04-17-1.xls' Attach (16/60)
 '2020-04-17-2.xls' Attach (17/60)
 '2020-04-17-3.xls' Attach (18/60)
 '2020-04-17-4.xls' Attach (19/60)
 '2020-04-17-5.xls' Attach (20/60)


D:\Anaconda\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


 '2020-04-18-1.xls' Attach (21/60)
 '2020-04-18-2.xls' Attach (22/60)
 '2020-04-18-3.xls' Attach (23/60)
 '2020-04-18-4.xls' Attach (24/60)
 '2020-04-18-5.xls' Attach (25/60)
 '2020-04-19-1.xls' Attach (26/60)
 '2020-04-19-2.xls' Attach (27/60)
 '2020-04-19-3.xls' Attach (28/60)
 '2020-04-19-4.xls' Attach (29/60)
 '2020-04-19-5.xls' Attach (30/60)
 '2020-04-20-1.xls' Attach (31/60)
 '2020-04-20-2.xls' Attach (32/60)
 '2020-04-20-3.xls' Attach (33/60)
 '2020-04-20-4.xls' Attach (34/60)
 '2020-04-20-5.xls' Attach (35/60)
 '2020-04-21-1.xls' Attach (36/60)
 '2020-04-21-2.xls' Attach (37/60)
 '2020-04-21-3.xls' Attach (38/60)
 '2020-04-21-4.xls' Attach (39/60)
 '2020-04-21-5.xls' Attach (40/60)
 '2020-04-22-1.xls' Attach (41/60)
 '2020-04-22-2.xls' Attach (42/60)
 '2020-04-22-3.xls' Attach (43/60)
 '2020-04-22-4.xls' Attach (44/60)
 '2020-04-22-5.xls' Attach (45/60)
 '2020-04-23-1.xls' Attach (46/60)
 '2020-04-23-2.xls' Attach (47/60)
 '2020-04-23-3.xls' Attach (48/60)
 '2020-04-23-4.xls' 

In [47]:
#P3
cdataU = DataFrame()
n = 1
for i in files3:
    data = pd.read_excel(i)
    data.drop({'序号'},axis=1,inplace=True) 
    cdataU = cdataU.append(data)
    print(' \'%s\' Attach (%s/%s)' % (i,n,len(files3)))
    n = n+1

 '2020-04-26-1.xls' Attach (1/50)
 '2020-04-26-2.xls' Attach (2/50)
 '2020-04-26-3.xls' Attach (3/50)
 '2020-04-26-4.xls' Attach (4/50)
 '2020-04-26-5.xls' Attach (5/50)
 '2020-04-27-1.xls' Attach (6/50)
 '2020-04-27-2.xls' Attach (7/50)
 '2020-04-27-3.xls' Attach (8/50)
 '2020-04-27-4.xls' Attach (9/50)
 '2020-04-27-5.xls' Attach (10/50)
 '2020-04-28-1.xls' Attach (11/50)
 '2020-04-28-2.xls' Attach (12/50)
 '2020-04-28-3.xls' Attach (13/50)
 '2020-04-28-4.xls' Attach (14/50)
 '2020-04-28-5.xls' Attach (15/50)
 '2020-04-29-1.xls' Attach (16/50)
 '2020-04-29-2.xls' Attach (17/50)
 '2020-04-29-3.xls' Attach (18/50)
 '2020-04-29-4.xls' Attach (19/50)
 '2020-04-29-5.xls' Attach (20/50)
 '2020-04-30-1.xls' Attach (21/50)
 '2020-04-30-2.xls' Attach (22/50)
 '2020-04-30-3.xls' Attach (23/50)
 '2020-04-30-4.xls' Attach (24/50)
 '2020-04-30-5.xls' Attach (25/50)
 '2020-05-01-1.xls' Attach (26/50)
 '2020-05-01-2.xls' Attach (27/50)
 '2020-05-01-3.xls' Attach (28/50)
 '2020-05-01-4.xls' Attach (2

In [14]:
Ulen = len(cdataU)
cdataU_dup = cdataU.drop_duplicates(subset=None, keep='first', inplace= False)
Ulen_dup = len(cdataU_dup)
print('重複筆數:',Ulen - Ulen_dup)
print('實用新型專利 總筆數(已刪重複):',Ulen_dup)

重複筆數: 122821
實用新型專利 總筆數(已刪重複): 3477179


In [15]:
cdataU_dup.index = np.arange(Ulen_dup)  ### 修改index

In [16]:
dupnumber = cdataU_dup[cdataU_dup.duplicated('公开（公告）号',keep='first')]['公开（公告）号']
lendup = len(dupnumber)
print('公开（公告）号 重複筆數:',lendup)

公开（公告）号 重複筆數: 3348


In [11]:
n=0
### 檢查哪個欄位不同 第n筆重複號
dupindex = cdataU_dup[cdataU_dup['公开（公告）号']==dupnumber.iloc[n]].index
for i in cdataU_dup.columns:
    if (cdataU_dup.loc[dupindex[0],'%s'%i]==cdataU_dup.loc[dupindex[1],'%s'%i]) == False:
        print('%s:'%i)
        print('     first:',cdataU_dup.loc[dupindex[0],'%s'%i])
        print('     second:',cdataU_dup.loc[dupindex[1],'%s'%i])

合享价值度:
     first: 7
     second: 8
优先权信息:
     first: nan
     second: nan
首次公开日:
     first: NaT
     second: 2019-08-23 00:00:00
CPC:
     first: nan
     second: nan
引证专利:
     first: nan
     second: nan
被引证专利:
     first: nan
     second: nan
被引证次数:
     first: nan
     second: nan
引证科技文献:
     first: nan
     second: nan
专利寿命（月）:
     first: nan
     second: nan
被引证国别(forward):
     first: nan
     second: nan
失效日:
     first: NaT
     second: NaT


In [17]:
### 只留最新一筆重複公開號
cdataU_dup = cdataU_dup.drop_duplicates('公开（公告）号',keep='last')
print('實用新型專利 總筆數(已刪重複公吿號):',len(cdataU_dup))

實用新型專利 總筆數(已刪重複公吿號): 3473831


In [18]:
cdataU_cn = cdataU_dup[cdataU_dup['申请人国别代码'] == '中国']
print(cdataU_cn.shape)

(3444021, 52)


In [53]:
cdataU_cn = cdataU_cn[~cdataU_cn['申请人省市代码'].isna()]
print(cdataU_cn.shape)

(2882953, 52)


In [19]:
cdataU_cn = cdataU_cn[~cdataU_cn['中国申请人区县'].isna()]
print(cdataU_cn.shape)
cdataU_cn = cdataU_cn[cdataU_cn['申请人类型'].str.contains('企业')]
print(cdataU_cn.shape)
cdataU_cn['申请年份'] = cdataU_cn['申请日'].map(lambda x:int(str(x)[0:4]))

(3246119, 52)
(2268124, 52)


In [20]:
applicant = cdataU_cn['申请人'].str.split(';',expand=True).stack().reset_index(level=1,drop=True).rename('申请人2')
cdataU_cn = cdataU_cn.join(applicant)
print(cdataU_cn.shape)
cdataU_cn = cdataU_cn[cdataU_cn['申请人2'].isin(app_list)]
print(cdataU_cn.shape)

(2469442, 54)
(1466338, 54)


In [56]:
cdataU_cn['申请人省市代码'].fillna('',inplace=True)
cdataU_cn['申请人省市代码'] = cdataU_cn['申请人省市代码'].map(lambda x:x.split(';')[0])
cdataU_cn['中国申请人地市'].fillna('',inplace=True)
cdataU_cn['中国申请人地市'] = cdataU_cn['中国申请人地市'].map(lambda x:re.sub(r"(\[)|(\])","",x))
cdataU_cn['中国申请人区县'].fillna('',inplace=True)
cdataU_cn['中国申请人区县'] = cdataU_cn['中国申请人区县'].map(lambda x:re.sub(r"(\[)|(\])","",x))
cdataU_cn['行政區'] = cdataU_cn.apply(lambda x:x['申请人省市代码']+x['中国申请人地市']+x['中国申请人区县'],axis=1)

In [57]:
#模糊比對行政區位置
cdataU_cn['校正'] = cdataU_cn.apply(lambda x:similaraity(city_list,x['申请人省市代码'],x['行政區']),axis=1)

In [58]:
cdataU_cn = pd.merge(cdataU_cn,city_code[['代碼','行政區']],left_on='校正',right_on='行政區',how='left')

In [26]:
cdataU_cn
##.to_csv(r'E:\paper\實用新型.csv')

,标题,文献页数,合享价值度,链接到incoPat,公开（公告）号,公开（公告）日,申请号,申请日,申请人,标准化申请人,申请人国别代码,申请人省市代码,中国申请人地市,中国申请人区县,申请人地址,申请人类型,优先权信息,权利要求数量,首次公开日,授权公告日,专利类型,发明人,IPC,CPC,简单同族,扩展同族,inpadoc同族,同族国家,简单同族个数,扩展同族个数,inpadoc同族个数,引证专利,被引证专利,被引证次数,引证科技文献,专利寿命（月）,被引证国别(forward),失效日,公开国别,申请年份,申请人2,行政區_x,校正,代碼,行政區_y
0,一种轻型起垄机,8,7,到incoPat中查看\r\nCN209861489U,CN209861489U,2019-12-31,CN201920577330.5,2019-04-25,长阳清江农机制造有限公司,NaN,中国,湖北,宜昌,长阳土家族自治县,443500 湖北省宜昌市长阳土家族自治县创新产业园科教路6号,企业,NaN,5.0,NaT,2019-12-31,实用新型,皮兆周; 周鹏; 吕正军,A01B33/02; A01B33/08; A01B33/10,NaN,CN209861489U,CN209861489U,CN209861489U,CN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,CN,2019,长阳清江农机制造有限公司,湖北宜昌长阳土家族自治县,湖北省宜昌市长阳土家族自治县,420528,湖北省宜昌市长阳土家族自治县
1,旋齿式农田废膜捡拾机,10,7,到incoPat中查看\r\nCN209861495U,CN209861495U,2019-12-31,CN201920400189.1,2019-03-27,甘肃洮河拖拉机制造有限公司,NaN,中国,甘肃,定西,临洮,730500 甘肃省定西市临洮县洮阳镇北五里铺路东1号,企业,NaN,10.0,NaT,2019-12-31,实用新型,马鹏程; 马明义; 马海军; 刘晓丽; 陈娟; 赵新平; 马进; 王亚茹; 孙多鑫; 崔彬彬; 李锦龙,A01B43/00,NaN,CN209861495U,CN209861495U,CN209861495U,CN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,CN,2019,甘肃洮河拖拉机制造有限公司,甘肃定西临洮,甘肃省定西市临洮县,621124,甘肃省定西市临洮县
2,挑膜齿做摆线运动的残膜捡拾机构,8,7,到incoPat中查看\r\nCN209861496U,CN209861496U,2019-12-31,CN201920400196.1,2019-03-27,甘肃洮河拖拉机制造有限公司,NaN,中国,甘肃,定西,临洮,730500 甘肃省定西市临洮县洮阳镇北五里铺路东1号,企业,NaN,9.0,NaT,2019-12-31,实用新型,马明义; 刘晓丽; 陈娟; 赵新平; 马海军; 马进; 王亚茹,A01B43/00,NaN,CN209861496U,CN209861496U,CN209861496U,CN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,CN,2019,甘肃洮河拖拉机制造有限公司,甘肃定西临洮,甘肃省定西市临洮县,621124,甘肃省定西市临洮县
3,一种园林养护装置,7,7,到incoPat中查看\r\nCN209861503U,CN209861503U,2019-12-31,CN201822181163.0,2018-12-25,广州科越装饰工程有限公司,NaN,中国,广东,广州,番禺,510000 广东省广州市番禺区大龙街东怡新区3街13号,企业,NaN,4.0,NaT,2019-12-31,实用新型,陈素玲,A01B49/06; A01B49/04,NaN,CN209861503U,CN209861503U,CN209861503U,CN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,CN,2018,广州科越装饰工程有限公司,广东广州番禺,广东省广州市番禺区,440113,广东省广州市番禺区
4,一种提高黄芪种子繁育率的设备,6,7,到incoPat中查看\r\nCN209861520U,CN209861520U,2019-12-31,CN201820293215.0,2018-03-02,陇西稷丰种业有限责任公司,NaN,中国,甘肃,定西,陇西,748100 甘肃省定西市陇西县首阳镇永宁路1号,企业,NaN,6.0,NaT,2019-12-31,实用新型,张双定; 周锐锋,A01C1/02,NaN,CN209861520U,CN209861520U,CN209861520U,CN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,CN,2018,陇西稷丰种业有限责任公司,甘肃定西陇西,甘肃省定西市陇西县,621122,甘肃省定西市陇西县
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123435,一种潜油直线电机无杆采油一体化装置,5,7,到incoPat中查看\r\nCN206376818U,CN206376818U,2017-08-04,CN201621379271.3,2016-12-15,中国石油天然气股份有限公司,[中国石油-C],中国,北京,北京,东城,100007 北京市东城区东直门北大街9号中国石油大厦,企业,NaN,5.0,NaT,2017-08-04,实用新型,杨海涛; 辛宏; 周志平; 甘庆明; 李佰涛; 张磊; 魏韦; 呼苏娟; 刘天宇,E21B34/06; E21B4/04,NaN,CN206376818U,CN206376818U,CN206376818U,CN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,CN,2016,中国石油天然气股份有限公司,北京北京东城,北京市东城区,110101,北京市东城区
1123436,一种电动压力平衡井下开关,10,9,到incoPat中查看\r\nCN206376819U,CN206376819U,2017-08-04,CN201621425751.9,2016-12-23,中国石油天然气集团公司; 中国石油集团测井有限公司,[中国石油-C],中国,北京,北京,东城,100007 北京市东城区东直门北大街9号,企业,NaN,9.0,NaT,2017-08-04,实用新型,陈永昌; 周德仁; 唐林; 黄美兰; 孟献军; 顾雪梅; 邓文传,E21B34/06; E21B43/14; E21B43/20,NaN,CN206376819U,CN206376819U,CN206376819U,CN,1,1,1,NaN,CN108222876A,1.0,NaN,NaN,中国,NaT,CN,2016,中国石油天然气集团公司,北京北京东城,北京市东城区,110101,北京市东城区
1123437,套管泄压装置、套管阀门及采油树,10,7,到incoPat中查看\r\nCN206376820U,CN206376820U,2017-08-04,CN201621383087.6,2016-12-16,中国石油天然气股份有限公司,[中国石油-C],中国,北京,北京,东城,100007 北京市东城区东直门北大街9号中国石油大厦,企业,NaN,10.0,NaT,2017-08-04,实用新型,姜许健; 王光辉; 昌伦杰; 陈文; 刘勇; 刘敏; 杨其展; 吴国磊; 王宏; 曹鹏; 郭志广; 任宏宇; 张识,E21B34/08,NaN,CN206376820U,CN206376820U,CN206376820U,CN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,CN,2016,中国石油天然气股份有限公司,北京北京东城,北京市东城区,110101,北京市东城区
1123438,双管环形空间的浮阀和双管装置,12,8,到incoPat中查看\r\nCN206376821U,CN206376821U,2017-08-04,CN201621406450.1,2016-12-21,中国石油天然气股份有限公司,[中国石油-C],中国,北京,北京,东城,100007 北京市东城区东直门北大街9号中国石油大厦,企业,NaN,9.0,NaT,2017-08-04,实用新型,盛勇; 黎强; 张卫贤; 张超; 赵闯; 康鹏; 秦世宽; 李剑波; 白彦明; 李斌; 熊志飚,E21B34/08; E21B17/00; E21B17/18; E21B17/10; E21B21/00,NaN,CN206376821U,CN206376821U,CN206376821U,CN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,CN,2016,中国石油天然气股份有限公司,北京北京东城,北京市东城区,110101,北京市东城区


In [59]:
#申請數
counts = cdataU_cn.groupby(['代碼','申请年份'])['申请号'].count()

In [60]:
#引用數
cdataU_cn['被引证次数'].fillna(0,inplace=True)
cdataU_cn['权利要求数量'].fillna(0,inplace=True)
cite = cdataU_cn.groupby(['代碼','申请年份'])[['被引证次数','权利要求数量']].sum()

In [29]:
#P1
patent_data1 = pd.merge(cite,counts,left_index=True,right_index=True,how='outer')

In [30]:
patent_data1

被引证次数   权利要求数量   申请号
代碼     申请年份                      
110101 2016   74.0   3613.0   430
       2017  295.0  28111.0  3200
       2018   37.0  30546.0  3480
       2019    0.0   3469.0   454
110102 2016  125.0   1994.0   320
...            ...      ...   ...
654324 2019    0.0      8.0     1
654325 2016    0.0     17.0     2
       2017    1.0    250.0    41
       2018    0.0    101.0    15
       2019    0.0     17.0     3

[7506 rows x 3 columns]

In [45]:
#P2
patent_data2 = pd.merge(cite,counts,left_index=True,right_index=True,how='outer')

In [46]:
patent_data2

被引证次数  权利要求数量   申请号
代碼     申请年份                      
110101 2012   680.0    2847   554
       2013  3281.0   14222  2866
       2014  2630.0   22289  3108
       2015  1971.0   24934  3340
       2016   916.0   24816  3077
...             ...     ...   ...
654325 2013   107.0     383    72
       2014    62.0     493    78
       2015    58.0     743   114
       2016    38.0     585    93
       2017     0.0       8     1

[10395 rows x 3 columns]

In [61]:
#P3
patent_data3 = pd.merge(cite,counts,left_index=True,right_index=True,how='outer')

In [62]:
patent_data3

被引证次数  权利要求数量  申请号
代碼     申请年份                    
110101 1985    0.0     3.0    1
       1987    4.0     1.0    1
       1988    1.0     2.0    1
       1990    2.0     3.0    1
       1991    3.0    14.0    5
...            ...     ...  ...
654325 2008   54.0   157.0   28
       2009   33.0   106.0   17
       2010   53.0   147.0   26
       2011   80.0   257.0   50
       2012   60.0   255.0   49

[15487 rows x 3 columns]

In [64]:
patent_data = pd.merge(patent_data1,patent_data2,left_index=True,right_index=True,how='outer')
patent_data = pd.merge(patent_data,patent_data3,left_index=True,right_index=True,how='outer')

In [66]:
patent_data.fillna(0,inplace=True)
patent_data['被引用次數'] = patent_data.apply(lambda x:x['被引证次数_x']+x['被引证次数_y']+x['被引证次数'],axis=1)
patent_data['權利要求數量'] = patent_data.apply(lambda x:x['权利要求数量_x']+x['权利要求数量_y']+x['权利要求数量'],axis=1)
patent_data['申請數'] = patent_data.apply(lambda x:x['申请号_x']+x['申请号_y']+x['申请号'],axis=1)

In [67]:
patent_data

被引证次数_x  权利要求数量_x  申请号_x  被引证次数_y  权利要求数量_y  申请号_y  被引证次数  \
代碼     申请年份                                                              
110101 1985      0.0       0.0    0.0      0.0       0.0    0.0    0.0   
       1987      0.0       0.0    0.0      0.0       0.0    0.0    4.0   
       1988      0.0       0.0    0.0      0.0       0.0    0.0    1.0   
       1990      0.0       0.0    0.0      0.0       0.0    0.0    2.0   
       1991      0.0       0.0    0.0      0.0       0.0    0.0    3.0   
...              ...       ...    ...      ...       ...    ...    ...   
654325 2015      0.0       0.0    0.0     58.0     743.0  114.0    0.0   
       2016      0.0      17.0    2.0     38.0     585.0   93.0    0.0   
       2017      1.0     250.0   41.0      0.0       8.0    1.0    0.0   
       2018      0.0     101.0   15.0      0.0       0.0    0.0    0.0   
       2019      0.0      17.0    3.0      0.0       0.0    0.0    0.0   

             权利要求数量  申请号  被引用次數  權利要求數量    申請數  
代碼     申请年份                                     
110101 1985     3.0  1.0    0.0     3.0    1.0  
       1987     1.0  1.0    4.0     1.0    1.0  
       1988     2.0  1.0    1.0     2.0    1.0  
       1990     3.0  1.0    2.0     3.0    1.0  
       1991    14.0  5.0    3.0    14.0    5.0  
...             ...  ...    ...     ...    ...  
654325 2015     0.0  0.0   58.0   743.0  114.0  
       2016     0.0  0.0   38.0   602.0   95.0  
       2017     0.0  0.0    1.0   258.0   42.0  
       2018     0.0  0.0    0.0   101.0   15.0  
       2019     0.0  0.0    0.0    17.0    3.0  

[29812 rows x 12 columns]

In [68]:
patent_data[['被引用次數','權利要求數量','申請數']].to_excel(r'E:\paper\實用新型_統計數量.xlsx')

# 合併兩者統計數量

In [ ]:
patentI = pd.read_excel(r'E:\paper\專利授權_統計數量.xlsx',index_col=[0,1])
patentI.reset_index(inplace=True)
patentI['代碼'] = patentI['代碼'].astype(str)
patentI = patentI.rename(columns={"代碼":"比對","申请年份":"year"})
patentU = pd.read_excel(r'E:\paper\實用新型_統計數量.xlsx',index_col=[0,1])
patentU.reset_index(inplace=True)
patentU['代碼'] = patentU['代碼'].astype(str)
patentU = patentU.rename(columns={"代碼":"比對","申请年份":"year"})

In [ ]:
patent = pd.merge(patentI,patentU,left_on=['比對','year'],right_on=['比對','year'],how='outer')
patent.fillna(0,inplace=True)
patent['總專利數'] = patent.apply(lambda x:x['申请号']+x['申請數'],axis=1)
patent['總引用數'] = patent.apply(lambda x:x['被引证次数']+x['被引用次數'],axis=1)
patent['總權利要求數'] = patent.apply(lambda x:x['权利要求数量']+x['權利要求數量'],axis=1)
patent.drop(['申请号','申請數','被引证次数','被引用次數','权利要求数量','權利要求數量'],axis=1,inplace=True)
patent = patent.sort_values(['比對','year']).reset_index(drop=True)

In [ ]:
patent['前一期'] = patent.groupby('比對')['year'].shift(1)

In [ ]:
for i in list(patent['比對'].unique()):
    counter = 1
    for k in list(patent[patent['比對']==i].index):
        if counter==1:
            patent.loc[k,'patent stock'] = patent.loc[k,'總專利數']
            counter = counter+1
        else:
            patent.loc[k,'patent stock'] = patent.loc[k-1,'patent stock']*0.85**(patent.loc[k,'year']-patent.loc[k,'前一期'])+patent.loc[k,'總專利數']
            counter = counter+1
            
for i in list(patent['比對'].unique()):
    counter = 1
    for k in list(patent[patent['比對']==i].index):
        if counter==1:
            patent.loc[k,'cite'] = patent.loc[k,'總引用數']
            counter = counter+1
        else:
            patent.loc[k,'cite'] = patent.loc[k-1,'cite']*0.85**(patent.loc[k,'year']-patent.loc[k,'前一期'])+patent.loc[k,'總引用數']
            counter = counter+1

for i in list(patent['比對'].unique()):
    counter = 1
    for k in list(patent[patent['比對']==i].index):
        if counter==1:
            patent.loc[k,'claim'] = patent.loc[k,'總權利要求數']
            counter = counter+1
        else:
            patent.loc[k,'claim'] = patent.loc[k-1,'claim']*0.85**(patent.loc[k,'year']-patent.loc[k,'前一期'])+patent.loc[k,'總權利要求數']
            counter = counter+1

In [ ]:
patent['cite stock'] = patent.apply(lambda x:x['cite']/x['patent stock'],axis=1)
patent['claim stock'] = patent.apply(lambda x:x['claim']/x['patent stock'],axis=1)
patent['year'] = patent['year'].astype(str)

In [ ]:
patent.to_excel(r'E:\paper\專利資料(授權+新型)_統計數量.xlsx')